In [28]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import RANSACRegressor
import time
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

california_housing = fetch_california_housing()
X = california_housing.data
y = california_housing.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

transformer = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)
y_train_transformed = transformer.fit_transform(y_train.reshape(-1, 1)).ravel()
y_test_transformed = transformer.transform(y_test.reshape(-1, 1)).ravel()

lasso_selector = SelectFromModel(Lasso(alpha=0.1))
X_train_selected = lasso_selector.fit_transform(X_train_scaled, y_train_transformed)
X_test_selected = lasso_selector.transform(X_test_scaled)

ransac_lasso = RANSACRegressor(base_estimator=Lasso(alpha=0.1), min_samples=100)
start_time = time.time()
ransac_lasso.fit(X_train_selected, y_train_transformed)
ransac_lasso_train_time = time.time() - start_time

ransac_lasso_train_pred = ransac_lasso.predict(X_train_selected)
ransac_lasso_test_pred = ransac_lasso.predict(X_test_selected)

ransac_lasso_train_pred = transformer.inverse_transform(ransac_lasso_train_pred.reshape(-1, 1)).ravel()
ransac_lasso_test_pred = transformer.inverse_transform(ransac_lasso_test_pred.reshape(-1, 1)).ravel()

ransac_lasso_train_mse = mean_squared_error(y_train, ransac_lasso_train_pred)
ransac_lasso_test_mse = mean_squared_error(y_test, ransac_lasso_test_pred)
ransac_lasso_train_r2 = r2_score(y_train, ransac_lasso_train_pred)
ransac_lasso_test_r2 = r2_score(y_test, ransac_lasso_test_pred)

ridge_model = Ridge()
start_time = time.time()
ridge_model.fit(X_train_scaled, y_train)
ridge_train_time = time.time() - start_time
ridge_train_pred = ridge_model.predict(X_train_scaled)
ridge_test_pred = ridge_model.predict(X_test_scaled)
ridge_train_mse = mean_squared_error(y_train, ridge_train_pred)
ridge_test_mse = mean_squared_error(y_test, ridge_test_pred)
ridge_train_r2 = r2_score(y_train, ridge_train_pred)
ridge_test_r2 = r2_score(y_test, ridge_test_pred)

lasso_model = Lasso()
lasso_params = {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0]}
lasso_grid = GridSearchCV(lasso_model, param_grid=lasso_params, scoring='neg_mean_squared_error', cv=5)
start_time = time.time()
lasso_grid.fit(X_train_scaled, y_train)
lasso_train_time = time.time() - start_time
lasso_best_model = lasso_grid.best_estimator_
lasso_train_mse = -lasso_grid.best_score_
lasso_test_mse = mean_squared_error(y_test, lasso_best_model.predict(X_test_scaled))
lasso_train_r2 = r2_score(y_train, lasso_best_model.predict(X_train_scaled))
lasso_test_r2 = r2_score(y_test, lasso_best_model.predict(X_test_scaled))

elasticnet_model = ElasticNet(max_iter=10000)
elasticnet_params = {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0], 'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]}
elasticnet_grid = GridSearchCV(elasticnet_model, param_grid=elasticnet_params, scoring='neg_mean_squared_error', cv=5)
start_time = time.time()
elasticnet_grid.fit(X_train_scaled, y_train)
elasticnet_train_time = time.time() - start_time
elasticnet_best_model = elasticnet_grid.best_estimator_
elasticnet_train_mse = -elasticnet_grid.best_score_
elasticnet_test_mse = mean_squared_error(y_test, elasticnet_best_model.predict(X_test_scaled))
elasticnet_train_r2 = r2_score(y_train, elasticnet_best_model.predict(X_train_scaled))
elasticnet_test_r2 = r2_score(y_test, elasticnet_best_model.predict(X_test_scaled))

poly_degree = 2
ridge_poly_model = make_pipeline(PolynomialFeatures(degree=poly_degree), Ridge(alpha=0.1))
start_time = time.time()
ridge_poly_model.fit(X_train_scaled, y_train)
ridge_poly_train_time = time.time() - start_time
ridge_poly_train_pred = ridge_poly_model.predict(X_train_scaled)
ridge_poly_test_pred = ridge_poly_model.predict(X_test_scaled)
ridge_poly_train_mse = mean_squared_error(y_train, ridge_poly_train_pred)
ridge_poly_test_mse = mean_squared_error(y_test, ridge_poly_test_pred)
ridge_poly_train_r2 = r2_score(y_train, ridge_poly_train_pred)
ridge_poly_test_r2 = r2_score(y_test, ridge_poly_test_pred)

print("Results:")
print("---------------------------------------------------------")
print("Linear Regression:")
print("  Training MSE: {:.4f}".format(linear_train_mse))
print("  Testing MSE: {:.4f}".format(linear_test_mse))
print("  Training R2: {:.4f}".format(linear_train_r2))
print("  Testing R2: {:.4f}".format(linear_test_r2))
print("  Training Time: {:.4f} seconds".format(linear_train_time))
print("---------------------------------------------------------")
print("RANSAC Regression with Lasso:")
print("  Training MSE: {:.4f}".format(ransac_lasso_train_mse))
print("  Testing MSE: {:.4f}".format(ransac_lasso_test_mse))
print("  Training R2: {:.4f}".format(ransac_lasso_train_r2))
print("  Testing R2: {:.4f}".format(ransac_lasso_test_r2))
print("  Training Time: {:.4f} seconds".format(ransac_lasso_train_time))
print("---------------------------------------------------------")
print("Ridge Regression:")
print("  Training MSE: {:.4f}".format(ridge_train_mse))
print("  Testing MSE: {:.4f}".format(ridge_test_mse))
print("  Training R2: {:.4f}".format(ridge_train_r2))
print("  Testing R2: {:.4f}".format(ridge_test_r2))
print("  Training Time: {:.4f} seconds".format(ridge_train_time))
print("---------------------------------------------------------")
print("Lasso Regression:")
print("  Training MSE: {:.4f}".format(lasso_train_mse))
print("  Testing MSE: {:.4f}".format(lasso_test_mse))
print("  Training R2: {:.4f}".format(lasso_train_r2))
print("  Testing R2: {:.4f}".format(lasso_test_r2))
print("  Training Time: {:.4f} seconds".format(lasso_train_time))
print("---------------------------------------------------------")
print("ElasticNet Regression:")
print("  Training MSE: {:.4f}".format(elasticnet_train_mse))
print("  Testing MSE: {:.4f}".format(elasticnet_test_mse))
print("  Training R2: {:.4f}".format(elasticnet_train_r2))
print("  Testing R2: {:.4f}".format(elasticnet_test_r2))
print("  Training Time: {:.4f} seconds".format(elasticnet_train_time))
print("---------------------------------------------------------")
print("Non-linear Regression (Ridge Regression with Polynomial Features)")
print("  Training MSE: {:.4f}".format(ridge_poly_train_mse))
print("  Testing MSE: {:.4f}".format(ridge_poly_test_mse))
print("  Training R2: {:.4f}".format(ridge_poly_train_r2))
print("  Testing R2: {:.4f}".format(ridge_poly_test_r2))
print("  Training Time: {:.4f} seconds".format(ridge_poly_train_time))
print("---------------------------------------------------------")


Results:
---------------------------------------------------------
Linear Regression:
  Training MSE: 0.5179
  Testing MSE: 0.5559
  Training R2: 0.6126
  Testing R2: 0.5758
  Training Time: 0.0049 seconds
---------------------------------------------------------
RANSAC Regression with Lasso:
  Training MSE: 0.9445
  Testing MSE: 0.9359
  Training R2: 0.2935
  Testing R2: 0.2858
  Training Time: 0.4157 seconds
---------------------------------------------------------
Ridge Regression:
  Training MSE: 0.5179
  Testing MSE: 0.5559
  Training R2: 0.6126
  Testing R2: 0.5758
  Training Time: 0.0106 seconds
---------------------------------------------------------
Lasso Regression:
  Training MSE: 0.5192
  Testing MSE: 0.5545
  Training R2: 0.6125
  Testing R2: 0.5769
  Training Time: 1.0886 seconds
---------------------------------------------------------
ElasticNet Regression:
  Training MSE: 0.5192
  Testing MSE: 0.5546
  Training R2: 0.6125
  Testing R2: 0.5768
  Training Time: 5.8121 s